In [ ]:
import numpy as np
import matplotlib as mpl
import pylab
from libtiff import TIFFfile
from libtiff import TIFF
import os
from scipy import interpolate
import cPickle

In [ ]:
def read(fileName):
    tiff = TIFFfile(fileName)
    samples, sample_names = tiff.get_samples()
    outList = []
    for sample in samples:
        outList.append(np.copy(sample)[...,np.newaxis])
        out = np.concatenate(outList,axis=-1)
        out = np.rollaxis(out,0,3)
        out = np.flipud(out)
        tiff.close()
    return out

In [ ]:
ImageData=read("../RameyMemoRaw.tiff")
Image=ImageData[:,:,0,0]

In [ ]:
pylab.imshow(Image,cmap='binary')
pylab.plot([1000,1000],[1000,4000],color='pink')

In [ ]:
#Put reference lines on for graphclick
pylab.imshow(Image,cmap='binary')
pylab.plot([2000,2000],[500,4000],color='pink')
pylab.plot([2000,9000],[500,500],color='pink')
pylab.plot([2000,9000],[500,500],color='pink')
pylab.savefig("Plots/WLines.png",dpi=200)

In [ ]:
TextLineFiles=os.listdir("TextLines/")
TextLineArrays=[]
pylab.imshow(Image,cmap='binary')
vars=np.arange(0,10000,10)
Padding=100
LimsLo=[]
LimsHi=[]
Funcs=[]
for t in sorted(TextLineFiles):
    ThisLine=np.loadtxt("TextLines/"+t,delimiter=',')
    TextLineArrays.append(ThisLine)
#    pylab.plot(ThisLine[:,0],ThisLine[:,1],'o',color='pink')
    func=interpolate.interp1d(ThisLine[:,0],ThisLine[:,1],bounds_error=False,fill_value="extrapolate")
    vars=np.arange(ThisLine[0][0],ThisLine[-1][0]+Padding,10)
    LimsLo.append(int(ThisLine[0][0]))
    LimsHi.append(int(ThisLine[-1][0])+Padding)
    Funcs.append(func)
    pylab.plot(vars,func(vars),'-',color='pink')
    pylab.ylim(0,5000)

In [ ]:
YScale=300
LineArrays=[]
for i in range(0,len(LimsLo)):
    ThisLineArray=np.ndarray(shape=(YScale,LimsHi[i]-LimsLo[i]))
    for a in range(LimsLo[i],LimsHi[i]):
        for x in range(0,YScale):
            ThisLineArray[x,a-LimsLo[i]]=Image[int(Funcs[i](a)-YScale/2+x),a]
    LineArrays.append(ThisLineArray)
    pylab.imshow(ThisLineArray,cmap='binary')
    pylab.show()

In [ ]:
ReversedOrder=[]
for i in range(0,len(LimsLo)):
    ReversedOrder.append(LineArrays[len(LimsLo)-i-1])

LineArraysFile=open("Pkls/LineArrays.pkl",'wb')
cPickle.dump(ReversedOrder,LineArraysFile)
LineArraysFile.close()

In [ ]:
NoiseLine=[[5500,10000],[2450,2100]]


pylab.imshow(Image,cmap='binary')
vars=np.arange(0,10000,10)
Padding=100
LimsLo=[]
LimsHi=[]
Funcs=[]
for t in sorted(TextLineFiles):
#    pylab.plot(ThisLine[:,0],ThisLine[:,1],'o',color='pink')
    noisefunc=interpolate.interp1d(NoiseLine[0],NoiseLine[1],bounds_error=False,fill_value="extrapolate")
    vars=np.arange(NoiseLine[0][0],NoiseLine[0][1]+Padding,10)
    LimsLo.append(int(NoiseLine[0][0]))
    LimsHi.append(int(NoiseLine[0][1])+Padding)
    pylab.plot(vars,noisefunc(vars),'-',color='white')
    pylab.ylim(0,5000)


In [ ]:
pylab.plot(vars,noisefunc(vars),'-',color='white')



In [ ]:
YScale=300
i=-1
NoiseLineArray=np.ndarray(shape=(YScale,LimsHi[i]-LimsLo[i]))
for a in range(LimsLo[i],LimsHi[i]):
    for x in range(0,YScale):
        NoiseLineArray[x,a-LimsLo[i]]=Image[int(noisefunc(a)-YScale/2+x),a]
pylab.figure(figsize=(15,1))


pylab.imshow(NoiseLineArray[:,0:3000],cmap='binary')
pylab.show()


In [ ]:
NoiseArraysFile=open("Pkls/NoiseArrays.pkl",'wb')
cPickle.dump(NoiseLineArray,NoiseArraysFile)
NoiseArraysFile.close()

In [ ]:
NoiseLineArray